# Reproduce paper figures with all noise spectra

In [ ]:
import time

from matplotlib import pyplot as plt
import numpy as np

import papermill as pm
from pdf2image import convert_from_path

In [ ]:
circuit_type: str = "qaoa"  # Supported types: "rb", "mirror", "qaoa".
nqubits: int = 2  # Number of qubits.
depth: int = 2  # Depth of circuit.
num_monte_carlo: int = 3000  # Number of samples to take in Monte carlo simulation.
number_of_experiments: int = 10  # Number of circuits to average over.
RUN_SIMULATIONS: bool = False  # Set "RUN_SIMULATIONS = False" for just plotting the results of pre-executed simulations.
fill_std = True  # Show std. dev. on final plot.
ymax = 0.4  # Largest vertical axis value on all four subplots in final plot.
save_final_figure = False # If True, the final figure is saved in a pdf file.

In [ ]:
parameters = {
    "circuit_type": circuit_type,
    "nqubits": nqubits,
    "depth": depth,
    "num_monte_carlo": num_monte_carlo,
    "save": True,
    "savefigs": False,
}

plot_parameters = {
    "fill_std": fill_std,
    "ymax": ymax
}

In [ ]:
noise_types = [
    {"noise_type": "white"},
    {"noise_type": "lowpass"},
    {"noise_type": "pink", "alpha": 1},
    {"noise_type": "pink", "alpha": 2},
]

## Run simulations

In [ ]:
if RUN_SIMULATIONS:
    init_time = time.time()

    for noise_params in noise_types:
        parameters.update(noise_params)
        for _ in range(number_of_experiments):
            _ = pm.execute_notebook(
                "run.ipynb", circuit_type + "_" + noise_params["noise_type"] + ".ipynb", parameters
            )

    print(f"Total execution time in minutes: {(time.time() - init_time) / 60 :.3f}")

## Produce individual plots

In [ ]:
parameters.update(plot_parameters)
for noise_params in noise_types:
    parameters.update(noise_params)
    _ = pm.execute_notebook(
        "plot.ipynb", "plot_" + circuit_type + "_" + noise_params["noise_type"] + ".ipynb", parameters
    )

## Combine plots into a unique figure

In [ ]:
dir_white = f"tzne_circuit_type_{circuit_type}_nqubits_{nqubits}_depth_{depth}_noise_type_white_base_power_0.01_None_max_scale_factor_9_num_monte_carlo_{num_monte_carlo}"
dir_lowpass = f"tzne_circuit_type_{circuit_type}_nqubits_{nqubits}_depth_{depth}_noise_type_lowpass_base_power_0.01_cutoff_0.01_max_scale_factor_9_num_monte_carlo_{num_monte_carlo}"
dir_pink1 = f"tzne_circuit_type_{circuit_type}_nqubits_{nqubits}_depth_{depth}_noise_type_pink_base_power_0.01_alpha_1.00_max_scale_factor_9_num_monte_carlo_{num_monte_carlo}"
dir_pink2 = f"tzne_circuit_type_{circuit_type}_nqubits_{nqubits}_depth_{depth}_noise_type_pink_base_power_0.01_alpha_2.00_max_scale_factor_9_num_monte_carlo_{num_monte_carlo}"

In [ ]:
# Convert pdf figures to png.
dpi = 600

top_left_png = convert_from_path(dir_white + "/000_errors.pdf", dpi)[0]
top_left = np.array(top_left_png)

top_right_png = convert_from_path(dir_lowpass + "/000_errors.pdf", dpi)[0]
top_right = np.array(top_right_png)

bottom_left_png = convert_from_path(dir_pink1 + "/000_errors.pdf", dpi)[0]
bottom_left = np.array(bottom_left_png)

bottom_right_png = convert_from_path(dir_pink2 + "/000_errors.pdf", dpi)[0]
bottom_right = np.array(bottom_right_png)

In [ ]:
plt.rcParams.update(
    {
        "text.usetex": True,
        "font.family": "serif",
        "font.size": 30,
        "font.weight": "bold",
    }
)
%matplotlib inline

fig, axs = plt.subplots(2, 2, figsize=(20, 10))

for ax in axs.flatten():
    ax.axis("off")

x = 600
y = -20
title_size = 25
label_size = 30

# (a) White noise.
axs[0, 0].imshow(top_left)
axs[0, 0].text(x, y, "(a)", fontsize=label_size)

# (b) Lowpass noise.
axs[0, 1].imshow(top_right)
axs[0, 1].text(x, y, "(b)", fontsize=label_size)

# (c) Pink 1/f noise.
axs[1, 0].imshow(bottom_left)
axs[1, 0].text(x, y, "(c)", fontsize=label_size)

# (d) Pink 1/f^2 noise.
axs[1, 1].imshow(bottom_right)
axs[1, 1].text(x, y, "(d)", fontsize=label_size)

plt.subplots_adjust(wspace=-0.30, hspace=0)

if save_final_figure:
    fig_name = f"error_figure_{circuit_type}_{nqubits}_{num_monte_carlo}_{number_of_experiments}.pdf"
    fig.savefig(fig_name)
